In [ ]:
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import MaxPooling2D,Convolution2D,Flatten,Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization

In [ ]:
model=Sequential()
model.add(Convolution2D(64,(3,3),strides=(1,1),use_bias=False,kernel_initializer='uniform',input_shape=(48,48,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.3))
model.add(MaxPooling2D())

In [ ]:
def conv_layer(model,filters,size):
    model.add(Convolution2D(filters,(size,size),strides=(1,1),use_bias=False,kernel_initializer='uniform'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    return model

In [ ]:
model=conv_layer(model,64,3)
#model=conv_layer(model,512,3)
#model=conv_layer(model,512,3)

model.add(Flatten())

In [ ]:
model.add(Dense(64,use_bias=False,kernel_initializer="uniform"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(64,use_bias=False,kernel_initializer="uniform"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(7,kernel_initializer="uniform",activation="softmax"))

In [ ]:
from keras import optimizers
adam=optimizers.adam(lr=0.01)
model.compile(optimizer=adam,loss="categorical_crossentropy",metrics=["accuracy"])

### for sample data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('sample_train',target_size=(48,48),batch_size=32,class_mode='categorical')
validation_generator = test_datagen.flow_from_directory('PrivateTest',target_size=(48,48),batch_size=32,class_mode='categorical')

In [ ]:

model.fit_generator(train_generator,steps_per_epoch=1875,epochs=3)

#### predicting sample data

In [ ]:
import pandas as pd
from keras.preprocessing import image
import numpy as np
import os

In [ ]:
pred_data=pd.read_csv("PublicTestData.csv")

In [ ]:
path=r"PublicTest"
result=[]
for i in range(3589):
    img=image.load_img(os.path.join(path,pred_data.iloc[i,3]),target_size=(48,48))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    res=list(model.predict(img)[0])
    result.append(res.index(max(res)))

In [ ]:
from sklearn.metrics import precision_score
print(precision_score(pred_data["Emotion"],result,average="weighted"))

In [ ]:
for i in result:
    print(result)

### for total data 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('Training',target_size=(48,48),batch_size=32,class_mode='categorical')
validation_generator = test_datagen.flow_from_directory('PrivateTest',target_size=(48,48),batch_size=32,class_mode='categorical')

In [ ]:

model.fit_generator(train_generator,steps_per_epoch=1875,epochs=3)

In [ ]:
test_set = test_datagen.flow_from_directory('PublicTests',
                                            target_size = (48, 48),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:

test_loss, test_acc = model.evaluate_generator(test_set)
print('test acc:', test_acc)



In [ ]:
path=r"PublicTest"
result=[]
for i in range(3589):
    img=image.load_img(os.path.join(path,pred_data.iloc[i,3]),target_size=(48,48))
    img=image.img_to_array(img)
    result.append(np.expand_dims(img,axis=0)[0])
result=np.array(result)
pred=model.predict_classes(result)

In [ ]:
from sklearn.metrics import precision_score,confusion_matrix,accuracy_score
pred_data=pd.read_csv("PublicTestData.csv")
print(precision_score(pred_data["Emotion"],pred,average="weighted"))
print(recall_score(pred_data["Emotion"],pred,average="weighted"))
print(accuracy_score(pred_data["Emotion"],pred))

In [ ]:
df=pd.DataFrame(pred)

In [ ]:
df[0].value_counts()

In [ ]:
print(confusion_matrix(pred_data["Emotion"],pred))

#### 56